**MAESTRÍA EN INTELIGENCIA ARTIFICIAL APLICADA**

**Curso: TC4029 - Ciencia y analítica de datos**

Tecnológico de Monterrey

Prof Grettel Barceló Alonso

**Semana 8**
Regresión Lineal

---

*   NOMBRE: ---------------------------------------------------------
*   MATRÍCULA: ----------------------------------------------------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
DIR = "/content/drive/MyDrive/Colab Notebooks/MNA/TC4029 - Ciencia y analítica de datos/Semana 8/Actividad8_RLineal"
os.chdir(DIR)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
insurance_df = pd.read_csv('insurance.csv')
insurance_df

# **Parte 1**. EDA

Efectúa una exploración inicial de los datos a través de:

1a) Estadísticas descriptivas para todas las variables del dataframe.

In [ ]:
display(insurance_df.describe())
insurance_df.select_dtypes(include=object).describe()

1b) Búsqueda de valores faltantes.

In [ ]:
insurance_df.isnull().mean()*100

1c) Histrogramas y boxplots para las variables cuantitativas y diagramas de barras con la frecuencia para las variables categóricas.

In [ ]:
#Cuantitativos
columnas = insurance_df.select_dtypes(exclude=object).columns
plt.figure(figsize=(15, 5))
for i, col in enumerate(columnas, 1):
    plt.subplot(1, 4, i)
    sns.boxplot(x=insurance_df[col])
    plt.title(f"Boxplot de {col}")
    plt.xlabel(col)
plt.tight_layout()

In [ ]:
insurance_df['sex'].index

In [ ]:
#Cualitativos
columnas = insurance_df.select_dtypes(include=object).columns
plt.figure(figsize=(15, 5))
for i, col in enumerate(columnas, 1):
    plt.subplot(1, 4, i)
    sns.barplot(insurance_df,x=insurance_df[col],y=insurance_df[col].count(), estimator="sum",errorbar=None)
    plt.title(f"Diagrama de barra de {col}")
    plt.xlabel(col)
plt.tight_layout()

Para responder los siguientes cuestionamientos, genera los gráficos indicados:

2a) ¿Hay diferencias significativas de las primas según el sexo? Compara la distribución de la variable `charges` por medio de un boxplot.

In [ ]:
plt.figure(figsize=(15, 5))
sns.boxplot(insurance_df,x=insurance_df['charges'],y=insurance_df['sex'])

2b) Crea un diagrama de dispersión de `charges` versus el `bmi` que incorpore la distinción por las categorías `smoker` y `children` (tip: utiliza el parámetro `size`) ¿Hay alguna conclusión importante que se derive del gráfico?

In [ ]:
sns.scatterplot(insurance_df,x=insurance_df['charges'],y=insurance_df['bmi'],size=insurance_df['smoker'])


2c) Elabora un mapa de calor que muestre la correlación de las variables numéricas.

In [ ]:
sns.heatmap(insurance_df.select_dtypes(exclude=object).corr())

Obtén un dataframe con las observaciones que tengan valores atípicos en `charges`.

3a) ¿Cuál es el promedio de `age` y de `bmi` en el dataframe de valores atípicos obtenido?

In [ ]:
l = insurance_df['charges'].quantile(0.25)
h = insurance_df['charges'].quantile(0.75)
iqr = h - l
ll = iqr - 1.5*iqr
hl = iqr + 1.5*iqr

atCharge_df = pd.DataFrame(insurance_df[(insurance_df['charges'] < ll) | (insurance_df['charges'] > hl)])
print('Promedio edad:',atCharge_df.age.mean(), 'y bmi:',atCharge_df.bmi.mean())

3b) ¿Qué porcentaje de estas observaciones (en el dataframe de valores atípicos obtenidos) se corresponden a mujeres y a hombres? ¿A fumadores y no fumadores?

In [ ]:
display(atCharge_df.sex.value_counts()/atCharge_df.shape[0]*100)
atCharge_df.smoker.value_counts()/atCharge_df.shape[0]*100

# **Parte 2**. Preprocesamiento

In [ ]:
atCharge_df.info()

4. Separa las variables del dataframe: en `X` coloca los predictores y en `y` la variable de respuesta o salida (`charges`).

In [ ]:
x = atCharge_df.drop(columns='charges')
y = atCharge_df.charges

5. Divide el conjunto en entrenamiento y prueba (80:20) considerando:


*   El parámetro `random_state` con el valor de 1 para garantizar reproducibilidad.
*   Analiza los diagramas de barras obtenidos en el ejercicio 1c. Con los resultados observados, es conveniente usar la variable `smoker` en la estrategia de estratificación.
*   Como ya tienes separadas en `X` e `y` los predictores y salida, consulta cómo ejecutar la función `train_test_split()` para obtener `Xtrain, Xtest, ytrain, ytest` en una única instrucción.

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.8, random_state=1)

6. Prepara un transformador, denominado `preprocessing`, para aplicar escalamiento *MinMax* a los predictores numéricos y codificación *one-hot* a los categóricas.

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

class Preprocessing(BaseEstimator, TransformerMixin):
  def __init__(self, df : pd.DataFrame):
     self.numData = df.select_dtypes(exclude=object)
     self.objData = df.select_dtypes(exclude=np.number)

  def fit(self):
    self.fitMinMax = MinMaxScaler.fit(self.numData)
    self.fitOneHot = OneHotEncoder.fit(self.objData)
    return self

  def transform(self):
    self.transMinMax = MinMaxScaler.fit(self.fitMinMax)
    self.transOneHot = OneHotEncoder.fit(self.fitOneHot)
    return self
# scaler = MinMaxScaler()
# ohe = OneHotEncoder(sparse_output=False,drop='first')
# x_sc = scaler.fit_transform(x.select_dtypes(exclude=object))
# x_ohe = ohe.fit_transform(x.select_dtypes(include=object))
preprocessing = make_pipeline(
    MinMaxScaler(),
    OneHotEncoder(handle_unknown="ignore"))

# **Parte 3**. Entrenando y evaluando el modelo de RL

7. Ejecuta un pipeline con el transformador creado y el algoritmo de regresión lineal para entrenar un modelo denominado `lr_model`.

Evalúa el modelo en el conjunto de prueba y obtén la raíz del error cuadrático medio (`RMSE`) y la métrica `R` cuadrado ($R^2$). Aunque ésta última la estudiarás a detalle la próxima semana, ahora sólo debes saber que toma valores entre 0 y 1, donde 1 indica un ajuste perfecto y, por tanto, un modelo muy fiable para las previsiones futuras. La función de scikit-learn que determina esta métrica es: [`r2_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, root_mean_squared_error

lr_model = LinearRegression()


**Valora el rendimiento del modelo con los siguientes cambios:**


8. Reemplaza los outliers de `charges` con el límite superior (`Q3 + 1.5 * IQR`). Haz el cambio en ambos conjuntos: `ytrain` y `ytest`, previo a la aplicación del mismo pipeline. Guarda el modelo como `lr_model2`.


9. Prepara otro transformador, denominado `preprocessing2`, para aplicar escalamiento estándar a los predictores numéricos y codificación ordinal a los categóricas. Aplícalo en un nuevo pipeline utilizando nuevamente regresión lineal. Guarda el modelo como `lr_model3`.

**Nota.** Los cambios se ejecutan de manera consecutiva, por ejemplo: el reemplazo de los outliers también se ejecuta previo al transformador `preprocessing2` y así sucesivamente.

10. Utiliza regresión polinomial para verificar si se produce un mejor ajuste. Intenta con varios grados, aunque debes dejar en el código sólo el de mejor rendimiento. Guarda el modelo como `lr_model4`.